In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
from pprint import pprint

boto_session = Session()
region = boto_session.region_name


agent_name = "deep_research_scoping_agent"
agent_core_client = boto3.client('bedrock-agentcore-control', region_name='us-west-2')

## Update SSM Parameter store from .env

In [ ]:
# Load .env file and create AWS Systems Manager secure parameters
from pathlib import Path
import dotenv

env_path = Path("/workspaces/agent_core_deep_research/.env")

if not env_path.exists():
    print(f"❌ .env file not found at {env_path}")
else:
    # Load environment variables
    env_vars = dotenv.dotenv_values(env_path)
    
    print(f"Found {len(env_vars)} environment variables in .env")
    
    # Create SSM client
    ssm_client = boto_session.client("ssm", region_name="us-west-2")
    
    # Create secure parameters for each key
    for key, value in env_vars.items():
        if not value or value.strip() == "":
            print(f"⚠️  Skipping empty value for {key}")
            continue
            
        parameter_name = f"/deep_research_scoping_agent/{key}"
        
        try:
            ssm_client.put_parameter(
                Name=parameter_name,
                Value=value,
                Type="SecureString",
                Overwrite=True,
                Description=f"Environment variable for deep_research_scoping_agent"
            )
            print(f"✅ Created/Updated secure parameter: {parameter_name}")
        except Exception as e:
            print(f"❌ Failed to create parameter {parameter_name}: {e}")
    
    print(f"\n✅ All environment variables stored as secure parameters in us-west-2")# Build Docker image and push to ECR


## Build and Push Docker to ECR

In [ ]:


import subprocess

# Get AWS account ID
account_id = boto_session.client("sts").get_caller_identity()["Account"]

# ECR repo URI
ecr_repo = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{agent_name}"

# Create ECR repository if it doesn't exist
print(f"Creating ECR repository: {agent_name}...")
ecr_client = boto_session.client("ecr")
try:
    ecr_client.create_repository(
        repositoryName=agent_name,
        imageScanningConfiguration={'scanOnPush': True}
    )
    print(f"✅ ECR repository '{agent_name}' created successfully")
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    print(f"ℹ️  ECR repository '{agent_name}' already exists")

# Authenticate Docker to ECR
print("Authenticating to ECR...")
login_password = subprocess.run(
    ["aws", "ecr", "get-login-password", "--region", region],
    capture_output=True,
    text=True,
    check=True
).stdout

subprocess.run(
    ["docker", "login", "--username", "AWS", "--password-stdin", f"{account_id}.dkr.ecr.{region}.amazonaws.com"],
    input=login_password,
    text=True,
    check=True
)

# Build Docker image (run from parent directory)
print(f"Building Docker image: {agent_name}...")
# subprocess.run(
#     ["docker", "build", "-f", "src/Dockerfile", "-t", agent_name, "."],
#     cwd="/workspaces/agent_core_deep_research",
#     check=True
# )

subprocess.run(
    ["docker", "build", "-t", agent_name, "."],
    check=True
)


# Tag Docker image
print(f"Tagging image for ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "tag", f"{agent_name}:latest", f"{ecr_repo}:latest"],
    check=True
)

# Push Docker image to ECR
print(f"Pushing image to ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "push", f"{ecr_repo}:latest"],
    check=True
)

print(f"✅ Docker image successfully pushed to {ecr_repo}:latest")

## Create Memory

In [33]:
from bedrock_agentcore.memory import MemoryClient

memory_name = "DeepResearchAgent"
client = MemoryClient(region_name=region)
memory = client.create_or_get_memory(name=memory_name)
memory_id = memory['id'] 

## Invoke by Boto3

In [85]:
#agent_name = "BasicAgentDemo_BasicAgent"
agent_name = "DeepResearchScopingAgent"
#agent_name = "deep_research_scoping_agent"
runtimeSessionId='dfmeoagmreaklgmrkleafremoigrmtesogmtrskhmtkrlshmt-g'  # Must be 33+ chars
thread_id='test-thread-35'

res = agent_core_client.list_agent_runtimes()
agent_id = [item['agentRuntimeId'] for item in res['agentRuntimes'] if item['agentRuntimeName']==agent_name][0]

res = agent_core_client.get_agent_runtime(agentRuntimeId=agent_id)
agentRuntimeArn = res['agentRuntimeArn']

agent_core_runtime_client = boto3.client('bedrock-agentcore')
payload = {
    "prompt": "I want to research the best coffee shops.",
    "thread_id": thread_id
}


In [86]:

response = agent_core_runtime_client.invoke_agent_runtime(
    agentRuntimeArn = agentRuntimeArn,
    runtimeSessionId=runtimeSessionId,
    payload=json.dumps(payload)
)

response_text = response['response'].read().decode('utf-8')
pprint(response_text)


('{"messages": [{"content": "I want to research the best coffee shops.", '
 '"additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": '
 'null, "id": "3897d579-7882-419e-91aa-4b68d9fd509f"}, {"content": "Could you '
 'please specify the location or city where you want to research the best '
 'coffee shops?", "additional_kwargs": {}, "response_metadata": {}, "type": '
 '"ai", "name": null, "id": "49f2027a-f9db-4150-9647-13bfea084e53", '
 '"tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}], '
 '"supervisor_messages": [], "raw_notes": [], "notes": []}')


In [83]:
json_res = json.loads(response_text)
pprint(json_res['messages'][-1]['content'])

('Could you please specify the location or city where you want to research the '
 'best coffee shops?')


In [84]:
payload1 = {
    "prompt": "Let's focus on Coffee quality in Seattle",
    "thread_id": thread_id
}

response = agent_core_runtime_client.invoke_agent_runtime(
    agentRuntimeArn = agentRuntimeArn,
    runtimeSessionId=runtimeSessionId,
    payload=json.dumps(payload1)
)

response_text = response['response'].read().decode('utf-8')
json_res = json.loads(response_text)
pprint(json_res)


{'messages': [{'additional_kwargs': {},
               'content': 'I want to research the best coffee shops.',
               'id': '3897d579-7882-419e-91aa-4b68d9fd509f',
               'name': None,
               'response_metadata': {},
               'type': 'human'},
              {'additional_kwargs': {},
               'content': 'Could you please specify the location or city where '
                          'you want to research the best coffee shops?',
               'id': '49f2027a-f9db-4150-9647-13bfea084e53',
               'invalid_tool_calls': [],
               'name': None,
               'response_metadata': {},
               'tool_calls': [],
               'type': 'ai',
               'usage_metadata': None},
              {'additional_kwargs': {},
               'content': "Let's focus on Coffee quality in Seattle",
               'id': '2fe18379-87cd-42d3-8eab-162207d02d0d',
               'name': None,
               'response_metadata': {},
               'typ

In [66]:
payload2 = {
    "prompt": "Let's focus on coffee bean origin and processing methods",
    "thread_id": "test-thread-2"
}

response = agent_core_runtime_client.invoke_agent_runtime(
    agentRuntimeArn = agentRuntimeArn,
    payload=json.dumps(payload2)
)

response_text = response['response'].read().decode('utf-8')
json_res = json.loads(response_text)
pprint(json_res)

{'messages': [{'additional_kwargs': {},
               'content': "Let's focus on coffee bean origin and processing "
                          'methods',
               'id': '64630a73-9331-4385-88d0-39d08a21c36f',
               'name': None,
               'response_metadata': {},
               'type': 'human'},
              {'additional_kwargs': {},
               'content': 'Thank you for specifying that you would like the '
                          'report to focus on coffee bean origin and '
                          'processing methods. I understand that the scope '
                          'should cover where coffee beans come from and the '
                          'various methods used to process them. I will now '
                          'begin the research based on this information.',
               'id': '7370d95c-a1d7-45f9-bd0c-b2b92f4719a1',
               'invalid_tool_calls': [],
               'name': None,
               'response_metadata': {},
             

## Read from Memory

In [77]:
import boto3
import json
from botocore.exceptions import ClientError

def list_events_with_payloads(
    memory_id: str,
    actor_id: str,
    session_id: str,   # <- your LangGraph thread_id
    *,
    region: str = "us-west-2",
    max_results: int = 50,
    limit_events: int | None = 200,
):
    ac = boto3.client("bedrock-agentcore", region_name=region)
    kwargs = {
        "memoryId": memory_id,
        "actorId": actor_id,
        "sessionId": session_id,
        "includePayloads": True,
        "maxResults": max_results,
    }
    res = ac.list_events(**kwargs)
    return res

MEMORY_ID = "DeepResearchAgent-7gZmvrCKCl"
ACTOR_ID  = "default_actor"      # your LangGraph 'actor_id'
THREAD_ID = "test-thread-1"   # your LangGraph 'thread_id'

res = list_events_with_payloads(MEMORY_ID, ACTOR_ID, THREAD_ID, region="us-west-2")
pprint(res)


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '178123',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 12 Nov 2025 20:48:43 GMT',
                                      'x-amzn-requestid': '0fc2f630-3d7c-4eb6-8ef0-0d9b114c2016'},
                      'HTTPStatusCode': 200,
                      'RequestId': '0fc2f630-3d7c-4eb6-8ef0-0d9b114c2016',
                      'RetryAttempts': 0},
 'events': [{'actorId': 'default_actor',
             'branch': {'name': 'main'},
             'eventId': '0000001762980463957#e6aa20bb',
             'eventTimestamp': datetime.datetime(2025, 11, 12, 20, 47, 43, 957000, tzinfo=tzlocal()),
             'memoryId': 'DeepResearchAgent-7gZmvrCKCl',
             'payload': [{'blob': '{"event_type": "channel_data", "channel": '
                                  '"branch:to:write_research_brief", '
      